<a href="https://colab.research.google.com/github/javierahermosa/AI/blob/main/RAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install --upgrade --quiet openai pypdf pydantic==1.10.8 scikit-learn langchain langchain[docarray] langchain-pinecone langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2319656 sha256=21d4b7a872e7992180a708810066d52a79af789e4b5256e7ecf25c777576d028
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [36]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import DocArrayInMemorySearch


from google.colab import userdata
import os

In [12]:
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['NEO4J_PASSWORD'] = userdata.get('NEO4J_PASSWORD')

filename = 'AI_Act.pdf'


## Model Setup
Let's define the LLM model we will use throughout the notebook.

In [14]:
model = ChatOpenAI(model="gpt-4o", temperature=0)

In [15]:
model.invoke("What are all the starwars movies")


AIMessage(content='As of my knowledge cutoff in October 2023, the "Star Wars" film series includes the following movies:\n\n### The Skywalker Saga\n1. **Episode I: The Phantom Menace** (1999)\n2. **Episode II: Attack of the Clones** (2002)\n3. **Episode III: Revenge of the Sith** (2005)\n4. **Episode IV: A New Hope** (1977)\n5. **Episode V: The Empire Strikes Back** (1980)\n6. **Episode VI: Return of the Jedi** (1983)\n7. **Episode VII: The Force Awakens** (2015)\n8. **Episode VIII: The Last Jedi** (2017)\n9. **Episode IX: The Rise of Skywalker** (2019)\n\n### Standalone Films (Anthology Series)\n1. **Rogue One: A Star Wars Story** (2016)\n2. **Solo: A Star Wars Story** (2018)\n\n### Animated Films\n1. **Star Wars: The Clone Wars** (2008)\n\nThese films are part of the official "Star Wars" canon and are produced by Lucasfilm. The Skywalker Saga is the main storyline, while the standalone films and animated film provide additional context and stories within the same universe.', response

## Adding a parser and creating our first simple chain


In [16]:
parser = StrOutputParser()

chain = model | parser
chain.invoke("What are all the starwars movies")

'As of my knowledge cutoff date in October 2023, here is a list of all the main Star Wars movies, including the episodic films and the standalone anthology films:\n\n### The Skywalker Saga (Episodes I-IX)\n1. **Episode I: The Phantom Menace** (1999)\n2. **Episode II: Attack of the Clones** (2002)\n3. **Episode III: Revenge of the Sith** (2005)\n4. **Episode IV: A New Hope** (1977)\n5. **Episode V: The Empire Strikes Back** (1980)\n6. **Episode VI: Return of the Jedi** (1983)\n7. **Episode VII: The Force Awakens** (2015)\n8. **Episode VIII: The Last Jedi** (2017)\n9. **Episode IX: The Rise of Skywalker** (2019)\n\n### Anthology Films (Standalone Films)\n1. **Rogue One: A Star Wars Story** (2016)\n2. **Solo: A Star Wars Story** (2018)\n\nThese films make up the core of the Star Wars cinematic universe. There are also numerous animated films, TV series, and other media that expand the Star Wars universe, but the above list includes the primary live-action films released in theaters.'

## Introducing prompt templates
We would like to create a prompt template that we can reuse to provide the model context in different forms

In [17]:
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="My favorite fruits are apples and pears.", question="What are my favorite fruits?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: My favorite fruits are apples and pears.\n\nQuestion: What are my favorite fruits?\n'

In [18]:
chain = prompt | model | parser
chain.invoke({
    "context": "My favorite fruits are apples and pears.",
    "question": "What are my favorite fruits?"
})

'Your favorite fruits are apples and pears.'

## Combining chains
This is useful when performing vaiorus fuctions, such as translations. We build a specific chain to translate the output of the first chain.

In [19]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [22]:
translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "My favorite fruits are apples and pears.",
        "question": "What are my favorite fruits?",
        "language": "German",
    }
)

'Deine Lieblingsfrüchte sind Äpfel und Birnen.'

## Load the AI act PDF and split it into documents

In [28]:
loader = PyPDFLoader(filename)
documents = loader.load()
documents[0:1]

[Document(page_content="ENUnited in diversityENEuropean Parliament\n2019-2024\nPlenary sitting\ncor01\n19.4.2024\nCORRIGENDUM\nto the position of the European Parliament adopted at first reading on 13 March 2024 with a view to \nthe adoption of Regulation (EU) 2024/ ...... of the European Parliament \nand of the Council laying down harmonised rules on artificial intelligence and amending \nRegulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) \n2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 \n(Artificial Intelligence Act)\nP9_TA(2024)0138 \n(COM(2021)0206 – C9-0146/2021 – 2021/0106(COD))\nIn accordance with Rule 241 of the European Parliament's Rules of Procedure, the above position is \ncorrected as follows:\nREGULATION (EU) 2024/… \nOF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nof …\nlaying down harmonised rules on artificial intelligence and amending Regulations (EC) No \n300/2008, (EU) No 167/2013, (EU) No 1

## Finding the Relevant Chunks
Here we introduce the idea of embeddings.

In [30]:
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("What are my favorite fruits?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[0.005407932660816378, -0.01827276677828038, 0.01719789759642708, -0.021570660096975447, -0.013093854253987184, 0.023708181371191805, -0.014974874390907887, -0.011976235503466322, 0.003038336046381708, -0.012849566480891484]


In [32]:
sentence1 = embeddings.embed_query("My favorite fruits are apples and pears.")
sentence2 = embeddings.embed_query("Women in tech know how to code.")

In [33]:
query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9134376275060934, 0.7083026768065336)

## Setting up a Vector Store
We start with an in-memory vector store

In [49]:
vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "My favorite fruits are apples and pears.",
        "My favorite summer fruits are peaches and cherries.",
        "Women in tech know how to code.",
        "Summers are better in Switzerland",
        "Thomas drives an old car",
        "Running is what I do in the morning",
    ],
    embedding=embeddings,
)

In [50]:
vectorstore1.similarity_search_with_score(query="What are my favorite fruits?", k=3)


[(Document(page_content='My favorite fruits are apples and pears.'),
  0.9134376361623296),
 (Document(page_content='My favorite summer fruits are peaches and cherries.'),
  0.8977980728829228),
 (Document(page_content='Running is what I do in the morning'),
  0.7671262360082354)]

## Connecting the vector store to the chain


In [51]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("What are my favorite fruits?")

[Document(page_content='My favorite fruits are apples and pears.'),
 Document(page_content='My favorite summer fruits are peaches and cherries.'),
 Document(page_content='Running is what I do in the morning'),
 Document(page_content='Summers are better in Switzerland')]

In [52]:
setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What are my favorite fruits?")

{'context': [Document(page_content='My favorite fruits are apples and pears.'),
  Document(page_content='My favorite summer fruits are peaches and cherries.'),
  Document(page_content='Running is what I do in the morning'),
  Document(page_content='Summers are better in Switzerland')],
 'question': 'What are my favorite fruits?'}

In [53]:
chain = setup | prompt | model | parser
chain.invoke("What are my favorite fruits?")

'Your favorite fruits are apples, pears, peaches, and cherries.'

In [54]:
chain.invoke("What do women in tech do?")

'Women in tech know how to code.'

## Loading the transcription into the vector store

In [57]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)
retriever2 = vectorstore2.as_retriever()

In [58]:
setup2 = RunnableParallel(context=retriever2, question=RunnablePassthrough())
chain = setup2 | prompt | model | parser

In [61]:
chain.invoke("Is the AI Act a USA Bill?")

'No, the AI Act is not a USA Bill. It is a regulation within the European Union.'

## Setting up Pinecone
So far we've used an in-memory vector store. In practice, we need a vector store that can handle large amounts of data and perform similarity searches at scale. For this example, we'll use Pinecone.

The first step is to create a Pinecone account, set up an index, get an API key, and set it as an environment variable PINECONE_API_KEY.

In [62]:
index_name = "myrag"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

In [63]:
pinecone.similarity_search("What is the AI Act?")[:3]


[Document(page_content='ENUnited in diversityEN(64) To mitigate the risks from high-risk AI systems placed on the market or put into service \nand to ensure a high level of trustworthiness, certain mandatory requirements should \napply to high-risk AI systems, taking into account the intended purpose and the context of \nuse of the AI system and according to the risk-management system to be established by the \nprovider. The measures adopted by the providers to comply with the mandatory \nrequirements of this Regulation should take into account the generally acknowledged \nstate of the art on AI, be proportionate and effective to meet the objectives of this \nRegulation. Based on the New Legislative Framework, as clarified in Commission notice \n“The ‘Blue Guide’ on the implementation of EU product rules 2022”, the general rule is \nthat more than one legal act of Union harmonisation legislation may be applicable to \none product, since the making available or putting into service can 

In [65]:
retriever3 = pinecone.as_retriever()
setup3 = RunnableParallel(context=retriever3, question=RunnablePassthrough())
setup3.invoke("What is the AI Act?")

{'context': [Document(page_content='ENUnited in diversityEN(64) To mitigate the risks from high-risk AI systems placed on the market or put into service \nand to ensure a high level of trustworthiness, certain mandatory requirements should \napply to high-risk AI systems, taking into account the intended purpose and the context of \nuse of the AI system and according to the risk-management system to be established by the \nprovider. The measures adopted by the providers to comply with the mandatory \nrequirements of this Regulation should take into account the generally acknowledged \nstate of the art on AI, be proportionate and effective to meet the objectives of this \nRegulation. Based on the New Legislative Framework, as clarified in Commission notice \n“The ‘Blue Guide’ on the implementation of EU product rules 2022”, the general rule is \nthat more than one legal act of Union harmonisation legislation may be applicable to \none product, since the making available or putting into 

In [67]:
chain3 = setup3 | prompt | model | parser
chain3.invoke("What is the AI Act?")

'The AI Act is a regulation aimed at improving the functioning of the internal market by establishing a uniform legal framework for the development, placing on the market, putting into service, and use of artificial intelligence (AI) systems in the European Union. It seeks to promote the uptake of human-centric and trustworthy AI while ensuring a high level of protection for health, safety, fundamental rights, democracy, the rule of law, and environmental protection. The regulation includes harmonized rules for AI systems, prohibitions of certain AI practices, and specific requirements for high-risk AI systems and obligations for their operators.'

## Testing
